In [ ]:
%reset

### Outline
1. [Load and transform the Adults dataset](#1)
2. [Create a model](#2)
3. [Create and Train the ART classifier](#3)
4. [Generate adversarial test examples](#4)
5. [Evaluate the model on benign and adversarial samples](#5)
    - [5.1 On Training Set](#5_1)
    - [5.2 On Test Set](#5_2)
    
- **Result**
    - Training score **100%** on Benign, **49%** on adversarial samples
    - Testing score **81%** on Benign, **37%** on adversarial samples

In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from metric import evaluate

import warnings
warnings.filterwarnings('ignore')

<a id="1"></a>
**1. Load and transform the Adults dataset**

In [2]:
from dataset import load_adults

# Step 1: Load the Adults dataset
(X_train, y_train), (X_test, y_test) = load_adults(True)
#X_train.shape (32561, 14)
#X_test.shape (16281, 14)
#y_train.shape (32561,)
#y_test.shape (16281,)

n_train = 100
n_test = 75
X_train = X_train[0:n_train]
y_train = y_train[0:n_train]
X_test = X_test[0:n_test]
y_test = y_test[0:n_test]
#X_train.shape (100, 14)
#y_train.shape (100, 2)



<a id="2"></a>
**2. Create the model**

In [3]:
from sklearn.tree import DecisionTreeClassifier

# Create the model
model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, 
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                               random_state=None, max_leaf_nodes=50, min_impurity_decrease=0.0, 
                               min_impurity_split=None, class_weight=None, presort=False)

# Create the model: SVC
# from sklearn.svm import SVC
# model = SVC(gamma='scale', probability=True, max_iter = -1, random_state=42)

<a id="3"></a>
**3. Create and Train the ART classifier**

In [4]:
from art.estimators.classification import SklearnClassifier

# Create the ART classifier
classifier = SklearnClassifier(model=model)

# Train the ART classifier
classifier.fit(X_train, y_train)

<a id="4"></a>
**4. Generate adversarial test examples**

In [5]:
from art.attacks.evasion import DecisionTreeAttack

# Create ART attack
attack = DecisionTreeAttack(classifier)

In [6]:
# Generate adversarial samples
X_train_adv = attack.generate(X_train)

Decision tree attack: 100%|█████████████████████████████████| 100/100 [00:00<00:00, 354.59it/s]


In [7]:
# Generate adversarial samples
X_test_adv = attack.generate(X_test)

Decision tree attack: 100%|███████████████████████████████████| 75/75 [00:00<00:00, 833.29it/s]


<a id="5"></a>
**5. Evaluate the model on benign and adversarial samples**
- [5.1 On Training Set](#5_1)
- [5.2 On Test Set](#5_2)

<a id="5_1"></a>
- 5.1. Evaluate the model on Training Set

In [8]:
print(f"Benign training:")
evaluate(classifier, X_train, y_train) #benign sample
# prediction = classifier.predict(X_train[0:1])[0]
# print("Benign Training Predicted Label: %i" % np.argmax(prediction))

Benign training:
y_pred: [0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0]
Misclassified examples:0
confusion matrix:
 [[75  0]
 [ 0 25]]
accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1: 1.0
auc: 1.0


In [9]:
print(f"Adversarial training:")
evaluate(classifier, X_train_adv, y_train) #Adversarial sample
# prediction = classifier.predict(X_train_adv[0:1, :])[0]
# print("Adversarial Training Predicted Label: %i"  % np.argmax(prediction))
# #prediction [1., 0.]

Adversarial training:
y_pred: [1 0 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1 0 0 1 1 1 0 0 1
 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 1 1
 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1]
Misclassified examples:51
confusion matrix:
 [[36 39]
 [12 13]]
accuracy: 0.49
Precision: 0.25
Recall: 0.52
F1: 0.33766233766233766
auc: 0.5


<a id="5_2"></a>
- 5.2. Evaluate the model on Test Set 

In [10]:
print(f"Benign testing:")
evaluate(classifier, X_test, y_test) #benign sample
# prediction = classifier.predict(X_test[0:1])[0]
# print("Benign Testing Predicted Label: %i" % np.argmax(prediction))

Benign testing:
y_pred: [0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1]
Misclassified examples:14
confusion matrix:
 [[49  7]
 [ 7 12]]
accuracy: 0.8133333333333334
Precision: 0.631578947368421
Recall: 0.631578947368421
F1: 0.631578947368421
auc: 0.7532894736842105


In [11]:
print(f"Adversarial testing:")
evaluate(classifier, X_test_adv, y_test) #adversarial sample
# prediction = classifier.predict(X_test_adv[0:1])[0]
# print("Adversarial Training Predicted Label: %i"  % np.argmax(prediction))
# #prediction [1., 0.]

Adversarial testing:
y_pred: [0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1
 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1
 1]
Misclassified examples:47
confusion matrix:
 [[25 31]
 [16  3]]
accuracy: 0.37333333333333335
Precision: 0.08823529411764706
Recall: 0.15789473684210525
F1: 0.11320754716981131
auc: 0.30216165413533835
